# Segmenting and Clustering Neighborhoods in Toronto 

## Assignments:

#### 1. Start by creating a new Notebook for this assignment.
#### 2. Use the Notebook to build the code to scrape from Wikipedia page 
#### 3. To create the above dataframe:      


     3.1  The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
     3.2  Ignore cells with a borough that is Not assigned.
     3.3  More than one neighborhood can exist in one postal code area. If a cell has a borough but a Not assigned neighborhood, 
          then the neighborhood will be the same as the borough. 
     3.3  Print the number of rows of your dataframe.

#### 4.  Get coordinates 

    4.1  Get coordinates for Toronto city from file Toronto_long_lat_data.csv in http://cocl.us/Geospatial_data
    4.2  Get the coordinates of each Neighborhood
    4.3  Get the coordinates of Toronto and plot the map
    4.4. Plot all Neighbourhood into Toronto map
    4.4. Plot selected Neighbourhood into Toronto map (Boroughs that contain the word 'Toronto')


#### 5.  Foursquare. Credentials and version

#### 6. Explore the first neighborhood in dataframe

    6.1  Get the neighborhood's coordinates 
    6.2  Get the top 100 venues within a radius of 500 meters
    6.3  Clean the JSON and structure it into a pandas dataframe   

#### 7. Explore neighborhoods

Note:	
		"DENC" = 	[D]owntown Toronto,
                    [E]ast Toronto, 
                    [N]orth Toronto, 
                    [C]entral Toronto, 


    7.1  Venues for each neighborhood
    7.2  Categories from all venues
    7.3  Analyze each neighborhood
    7.4  Cluster neighborhoods
    7.5  Examine clusters
    
        7.5.1  Clusters 1
        7.5.2  Clusters 2
        7.5.3  Clusters 3
        7.5.4  Clusters 4
        7.5.5  Clusters 5

## 1. Lib

In [1]:
import wikipedia as wp
import requests                                  # library to handle requests
import pandas as pd                              # library for data analsysis
import numpy as np                               # library to handle data in a vectorized manner
import matplotlib.cm as cm
import matplotlib.colors as colors
import random                                    # library for random number generation
import folium                                    # plotting library
import json
from geopy import Nominatim                      # module to convert an address into latitude and longitude values
from IPython.display import Image                # libraries for displaying images
from IPython.core.display import HTML            # libraries for displaying images
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize        # tranforming json file into a pandas dataframe library


## 2. Scraping a Wikipedia table using Panda in Python 
#### (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [2]:
#Get the html source
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)

## 3. Dataframe

### 3.1  Dataframe ( Three columns: PostalCode, Borough, and Neighborhood)

In [3]:
dfr = df.rename(columns={0:'PostalCode', 1:'Borough', 2: 'Neighborhood'})
dfd = dfr.drop([0,0,0], axis=0)
dfd.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### 3.2 Filtering Dataframe. Ignoring cells with a Borough that is 'Not assigned'.

In [4]:
dff = dfd[dfd['Borough'] != 'Not assigned']
dff.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### 3.3 Grouping By PostCode/Borough and concatenate Neighbourhood string names

In [5]:
dfg = dff.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
dfg.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


###  3.4  Dataframe shape result

In [6]:
print ("Dataframe shape: ", dfg.shape)


Dataframe shape:  (103, 3)


## 4.  Get coordinates


### 4.1  Get coordinates for Toronto city from file Toronto_long_lat_data.csv in  http://cocl.us/Geospatial_data

##### Note: We were not able to get coordinates of the neighborhoods  using the Geocoder package

In [7]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
dfp = pd.read_csv('Toronto_long_lat_data.csv')
dfp.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 4.2  Get the coordinates of each Neighborhood

In [8]:
dft = pd.merge(dfg, dfp, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
dft.drop("PostalCode", axis=1, inplace=True)
dft.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### 4.3 Get the coordinates of Toronto and plot the map

In [9]:
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto:  {}, {}'.format(latitude, longitude))
# create map of Toronto using latitude and longitude values
toronto_m = folium.Map(location=[latitude, longitude], zoom_start=10)
toronto_m

Coordinates of Toronto:  43.653963, -79.387207


### 4.4  Plot all Neighbourhood into Toronto map  

In [10]:
for lat, lng, Borough, Neighborhood in zip(dft['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_m)  
toronto_m

In [11]:
dftc = dft[dft['Borough'].str.contains("Toronto")].reset_index(drop=True)
dftc

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
5,Central Toronto,Davisville North,M4P,43.712751,-79.390197
6,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
7,Central Toronto,Davisville,M4S,43.704324,-79.388790
8,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160
9,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V,43.686412,-79.400049


### 4.4  Plot selected Neighbourhood into Toronto map (Boroughs that contain the word 'Toronto')

In [12]:
toronto_mc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, Borough, Neighborhood in zip(dftc['Latitude'], dftc['Longitude'], dftc['Borough'], dftc['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_mc)  
toronto_mc

### 5.  Foursquare. Credentials and version

In [13]:
CLIENT_ID = 'QL2OMYOY1B24UKBMDUGFY0HY0IKMSQIUCIPLHRK334LUQYZZ'
CLIENT_SECRET = 'K5KVFRWRKBQFTESD24TEQVG2EBWC4DCGO1HYHNS0WUOPFDTA' 
VERSION = '20180604'
LIMIT = 30

## 6. Explore the first neighborhood in dataframe


### 6.1 Get the neighborhood's coordinates 

In [14]:
neighborhood_n = dftc.loc[0, 'Neighborhood']
neighborhood_lat = dftc.loc[0, 'Latitude']
neighborhood_lon = dftc.loc[0, 'Longitude']



### 6.2  Get the top 100 venues within a radius of 500 meters

In [15]:
# defining radius and limit of venues to get
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_lat, neighborhood_lon, radius, LIMIT)
# get the result to a json file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d11ceaaaf35f300254ec85f'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

### 6.3 Clean the JSON and structure it into a pandas dataframe  

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0: return None
    else:
        return categories_list[0]['name']


In [17]:
venues = results['response']['groups'][0]['items']

# flatten JSON
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


### 7. Explore neighborhoods
####   Note:  "DENC" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    
    return(nearby_venues)

### 7.1 Venues for each neighborhood

In [19]:
toronto_v = getNearbyVenues(names = dftc['Neighborhood'], latitudes = dftc['Latitude'], longitudes = dftc['Longitude'])
                                  

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [20]:
toronto_v.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [21]:
toronto_v.shape


(1700, 7)

### 7.2 Categories from all venues

In [22]:
toronto_v.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


In [23]:
print('There are {} uniques categories.'.format(len(toronto_v['Venue Category'].unique())))

There are 238 uniques categories.


### 7.3 Analyze each neighborhood

In [24]:
# one hot encoding
toronto_oh = pd.get_dummies(toronto_v[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_oh['Neighborhood'] = toronto_v['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_oh.columns[-1]] + list(toronto_oh.columns[:-1])
toronto_oh.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_oh.shape

(1700, 238)

In [26]:
toronto_gr = toronto_oh.groupby('Neighborhood').mean().reset_index()
toronto_gr.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.04,...,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.0,0.000000
1,Berczy Park,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.018182,0.0,0.0,0.0,0.00,0.0,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000


In [27]:
def common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


In [30]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try: 
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Neighborhood'] = toronto_gr['Neighborhood']
       
for ind in np.arange(toronto_gr.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = common_venues(toronto_gr.iloc[ind, :], num_top_venues)
    

In [31]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Steakhouse,Thai Restaurant,Cosmetics Shop,Restaurant,Hotel,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Café,Italian Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Convenience Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Pizza Place,Brewery,Skate Park,Burrito Place,Butcher,Restaurant,Recording Studio,Comic Shop,Spa
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Boat or Ferry,Boutique,Plane,Bar


### 7.4 Cluster neighborhoods

In [34]:
# Run k-means to cluster the neighborhood into 2 clusters.
# number of clusters
kclusters = 5
toronto_denc_gc = toronto_gr.drop('Neighborhood', 1)
# run k-means 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_gc)
# check cluster labels 
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Steakhouse,Thai Restaurant,Cosmetics Shop,Restaurant,Hotel,Burger Joint
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Café,Italian Restaurant,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery
2,0,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Convenience Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
3,0,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Pizza Place,Brewery,Skate Park,Burrito Place,Butcher,Restaurant,Recording Studio,Comic Shop,Spa
4,0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Boat or Ferry,Boutique,Plane,Bar


In [36]:
toronto_denc_m = dftc
toronto_denc_m.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [37]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_m = toronto_denc_m.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_denc_m.head() # check the last columns

,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Other Great Outdoors,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Bakery,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572,0,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,East Toronto,Studio District,M4M,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Brewery,Seafood Restaurant,Sandwich Place
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [38]:
toronto_denc_m.head()


,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Other Great Outdoors,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Bakery,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572,0,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,East Toronto,Studio District,M4M,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Brewery,Seafood Restaurant,Sandwich Place
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
    toronto_denc_m['Latitude'],
    toronto_denc_m['Longitude'],
    toronto_denc_m['Neighborhood'],
    toronto_denc_m['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)

    folium.CircleMarker(
                    [lat, lon],
                    radius = 5,
                    popup = label,
                    color = rainbow[cluster-1],
                    fill = True,
                    fill_color = rainbow[cluster-1],
                    fill_opacity=0.7).add_to(map_clusters)
map_clusters


#### 7.5.1 Clusters 1

In [47]:
toronto_denc_m.loc[toronto_denc_m['Cluster Labels'] == 0, toronto_denc_m.columns[[1] + list(range(5, toronto_denc_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Health Food Store,Trail,Pub,Other Great Outdoors,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Bakery,Fruit & Vegetable Store,Juice Bar,Liquor Store
2,"The Beaches West, India Bazaar",0,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,Studio District,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Brewery,Seafood Restaurant,Sandwich Place
5,Davisville North,0,Playground,Hotel,Clothing Store,Food & Drink Shop,Park,Sandwich Place,Grocery Store,Gym,Breakfast Spot,Event Space
6,North Toronto West,0,Coffee Shop,Yoga Studio,Bagel Shop,Park,Clothing Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Diner,Salon / Barbershop
7,Davisville,0,Pizza Place,Dessert Shop,Sandwich Place,Thai Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Italian Restaurant,Café,Farmers Market
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Pub,Fried Chicken Joint,Pizza Place,Bagel Shop,Vietnamese Restaurant,Sushi Restaurant,Liquor Store,Supermarket,Light Rail Station
11,"Cabbagetown, St. James Town",0,Coffee Shop,Restaurant,Pizza Place,Café,Bakery,Pub,Park,Italian Restaurant,Japanese Restaurant,Breakfast Spot
12,Church and Wellesley,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Gay Bar,Men's Store,Gym,Pub,Mediterranean Restaurant,Bubble Tea Shop


#### 7.5.2 Clusters 2

In [53]:
toronto_denc_m.loc[toronto_denc_m['Cluster Labels'] == 1, toronto_denc_m.columns[[1] + list(range(5, toronto_denc_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",1,Playground,Tennis Court,Concert Hall,Convenience Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### 7.5.3 Clusters 3

In [54]:
toronto_denc_m.loc[toronto_denc_m['Cluster Labels'] == 2, toronto_denc_m.columns[[1] + list(range(5, toronto_denc_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,2,Park,Playground,Trail,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
23,"Forest Hill North, Forest Hill West",2,Jewelry Store,Trail,Sushi Restaurant,Park,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio


#### 7.5.4 Clusters 4

In [55]:
toronto_denc_m.loc[toronto_denc_m['Cluster Labels'] == 3, toronto_denc_m.columns[[1] + list(range(5, toronto_denc_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,3,Ice Cream Shop,Garden,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### 7.5.4 Clusters 5

In [56]:
toronto_denc_m.loc[toronto_denc_m['Cluster Labels'] == 4, toronto_denc_m.columns[[1] + list(range(5, toronto_denc_m.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,4,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
